In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#userDirectory = "/content/drive/MyDrive/CS465/CS465_final_project/"
userDirectory = "C:/Users/seanm/OneDrive - MNSCU/Fall 2021/CS 465/CS465_final_project"

In [ ]:
#userDirectory = "/content/drive/MyDrive/CNN/"

#Import Modules

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

#Helper Functions

In [ ]:
##Reads image, resizes images to size passed in, then converts to a numpy array
def read_image(path, size):
    import cv2
    import numpy as np
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (size, size))
    image = image.astype(np.float32)
    return image

#Test Model

In [ ]:
path = userDirectory
train_path = os.path.join(path, "datasets/Images/**/*")
labels_path = os.path.join(path, "datasets/labels_full.csv")


labels_df = pd.read_csv(labels_path)
breed = labels_df["breed"].unique()
print("Number of Breed: ", len(breed))

breed2id = {name: i for i, name in enumerate(breed)}
id2breed = {i: name for i, name in enumerate(breed)}

ids = glob(train_path)
labels = []

for image_id in ids:
    image_id = image_id.replace("\\","/")
    image_id = image_id.split("/")[-1].split(".")[0].split("-")[0]
    breed_name = list(labels_df[labels_df.id == image_id]["breed"])[0]
    breed_idx = breed2id[breed_name]
    labels.append(breed_idx)

#ids = ids[:1000]
#labels = labels[:1000]

## Spliting the dataset
train_x, valid_x = train_test_split(ids, test_size=0.1, random_state=0)
train_y, valid_y = train_test_split(labels, test_size=0.1, random_state=0)

## Model
checkpoint_path = os.path.join(path, "datasets/DenseNet201.h5")
model = tf.keras.models.load_model(checkpoint_path)
denominator = 50
numerator = 0
for i, path in tqdm(enumerate(valid_x[:denominator])):
    image = read_image(path, 224)
    image = np.expand_dims(image, axis=0)
    pred = model.predict(image)[0]
    label_idx = np.argmax(pred)
    breed_name = id2breed[label_idx]

    ori_breed = id2breed[valid_y[i]]
    ori_image = cv2.imread(path, cv2.IMREAD_COLOR)

    #ori_image = cv2.rectangle(ori_image, (0,0), (300,45), (255,255,255), -1)
    #ori_image = cv2.putText(ori_image, "Predicted: " + breed_name, (5, 15), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 0), 1)
    #ori_image = cv2.putText(ori_image, "Real: " + ori_breed, (5, 35), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 0), 1)
    #cv2.imwrite("C:/Users/seanm/Downloads/save/testing_"+ str(i) + ".png", ori_image)

    # plt.imshow(cv2.cvtColor(ori_image, cv2.COLOR_BGR2RGB))
    # plt.show()

    if(breed_name == ori_breed):
      numerator += 1
print("\nTesting accuracy: " + str((numerator/denominator)*100) + "%")

Number of Breed:  120


50it [00:12,  3.97it/s]


Testing accuracy: 96.0%
